In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes, matplotlib.ticker
%matplotlib inline
plt.ioff()

In [2]:
from glob import glob
from datetime import datetime
from itertools import *
from operator import itemgetter
import json, sys, traceback, os

In [3]:
spec_files = glob('profiling/*/specs')
spec_files.sort()
specs = {os.path.basename(os.path.dirname(f)):np.genfromtxt(f, delimiter=' ', dtype=int)
         for f in spec_files}

In [4]:
linestyles = ["-","--","-.",":"]
colors = ('blue','black','red','orange','green','magenta','yellow', 'cyan', 'purple', 'firebrick')
colorcycle = cycle(colors)
lastpe = False
line_def = []
color_def = []
## Currently disabled due to fragile and unused output
if False:
    for spec in specs:
        pe, comp, impl, mode = (spec[2], spec[4], spec[5], spec[6])
        if not lastpe or lastpe != pe:
            color = next(colorcycle)
            linecycle = cycle(linestyles)
        line  = next(linecycle)
        lastpe = pe
        line_def.append(line)
        color_def.append(color)

In [5]:
def set_keys(*indices):
    """Returns a function that returns a tuple of key values"""
    def get_keys(seq, indices=indices):
        keys = []
        for i in indices:
            keys.append(seq[i])
        return tuple(keys)
    return get_keys

In [6]:
def get_average_times(dat, dirCol, stepCol, timeCol):
    if dat.size == 1:
        return np.array([np.array([dat.item()[timeCol]])])
    dat.sort(order=['dir', 'step'])
    result = {}
    for (c_dir, c_step), rows in groupby(dat, key=set_keys('dir','step')):
        #print c_dir, c_step, rows
        c_dir, c_step, total_time, counter = None, None, 0, 0
        for row in rows:
            if not c_dir:
                c_dir = row[dirCol]
            if not c_step:
                c_step = row[stepCol]
            total_time+=row[timeCol]
            counter += 1
        result.setdefault(c_dir, []).append([c_step, total_time*1.0/counter])
    result2 = []
    for c_dir, rows in result.iteritems():
        inner = []
        result2.append([row[1] for row in rows])
#        for row in rows:
#            inner.append(row[1])
#        result2.append(inner)
    return np.array([np.array(rows) for rows in result2])

In [7]:
def is_axis(a):
    return not type(a) is np.ndarray

In [8]:
with open('profiling/kinds.json') as fd:
    kinds = json.load(fd)
print 'Kinds: {}'.format(kinds)

Kinds: {u'strategies': [u'normal', u'flush', u'noncached'], u'changes': [u'update', u'sw', u'res', u'complex']}


In [9]:
no_split, split_change_only, split_both = 0, 1, 2
dat, dat2, dat3 = None, None, None
def read_single_result(f, name, dtype, data_column, since):
    global dat, dat2, dat3
    def convdate(text):
        return datetime.strptime(text, '%Y-%m-%dT%H:%M:%S.%f')
    def convdir(text):
        return int(text[-3:])
    def convstep(text):
        return int(text[0:2])
    def safe_div(a, b):
        return a/b if b>0 else "{}!".format(a)
    dat = np.genfromtxt(f, delimiter=',', names=True, dtype=dtype,
                        converters={'timestamp':convdate, 'step':convstep, 'dir': convdir})
    if since:
        dat = dat[dat['timestamp'] > since ]
    dat2 = get_average_times(dat, 1, 2, data_column).transpose()
    len_dat = 1 if len(dat.shape) == 0 else len(dat)
    if dat2.size == 0:
        print 'Did not load any record for {}'.format(name)
    else:
        runs = '~= {} run(s)'.format(safe_div(len_dat,dat2.size)) if len_dat > 1 else '- no data'
        print 'Loaded {0} record(s) for {1} ({2[0]}x{2[1]} unique) {3}'.format(len_dat, name, dat2.shape, runs)
    dat3 = dat2
    return dat2

def read_results(prefix, name, dtype, data_column, since, splitted = no_split):
    if splitted in (split_change_only, split_both):
        result = {}
        for change in kinds['changes']:
            result.setdefault(change, {})
            if splitted == split_both:
                for strategy in kinds['strategies']:
                    result[change].setdefault(strategy, {})
                    new_name = '{0}_{1}_{2}'.format(change, strategy, name)
                    f = 'profiling/splitted/{0}_{1}.csv'.format(prefix, new_name)
                    result[change][strategy] = read_single_result(f, new_name, dtype, data_column, since)
            else: # splitted == split_change_only
                new_name = '{0}_{1}'.format(change, name)
                f = 'profiling/splitted/{0}_{1}.csv'.format(prefix, new_name)
                result[change] = read_single_result(f, new_name, dtype, data_column, since)
        return result
    else: # splitted = no_split
        f = 'profiling/{0}-{1}-results.csv'.format(prefix, name)
        return read_single_result(f, name, dtype, data_column, since)

In [10]:
def read_gen_results(impl, since = None):
    return read_results('gen', impl, ('datetime64[us]', int, int, float), 3, since, splitted = split_both)
def read_sol_results(solver, since = None):
    return read_results('sol', solver, ('datetime64[us]', int, int, int, int, int, float, float), 7, since,
                        splitted = split_change_only)

In [11]:
def safe(a, i, start = 0):
    try:
        return a[start:,i]
    except IndexError:
        return np.zeros(a[:,0].size)

def es(y, x):
    """ Ensure size of y compared to x """
    if y.shape[0] != x.shape[0]:
        y = np.ones(x.shape[0]) * y[0]
    return y

## ILP-Re-Generation and Solving Time

- ILP-Re-Generation describes the time it takes for the Scheme implementations to evaluate the attribute computing the ILP
    - it does not include the time to read from and write to disk
    - "Java" denotes the Java- and EMF-based generation
- ILP Solving describes the time it takes the solvers (GLPK and Gurobi in this case) to solve the generated ILP
    - "GLPK (Java)" denotes the time to solve the ILP generated by the Java-based generation with GLPK. Its format was modifiert slightly to be accepted by GLPK, as it was originally generated for lp_solve
    - "GLPK (Scheme)" denotes the time to solve the ILP generated by any scheme implementation with GLPK
- the plots show different system configurations
    - a system configuration is given by "r x ( c \* i \* m )", which describes a system with *r* resources and *c* software components with *i* implementations having *m* modes each.
- for one such configuration the same experiment is run, i.e. the system is modified 7 times leading to the 7 steps, whereas only changes on hardware resources are made
    - for the cases involving Java (both, generation and solving), the same time is used for every step, as the generation always starts from skretch. Further, the changes (e.g. change the value of a single hardware resource while generation) can not be reflected by the Java System Generator
- on the x-axis, the steps of this manipulation are shown
    - the initial generation of the ILP (step zero) is only shown below in numbers, as it would skew the diagrams

Header = ['timestamp', 'dir', 'step', 'rows', 'cols', 'non-zero', 'ilp-sol', 'ti-ilp-sol']

In [12]:
racket_dats  = read_gen_results('plt-r6rs', since = datetime(2015,8,17,0,0,0))
larceny_dats = read_gen_results('larceny', since = datetime(2015,6,12,0,0,0))
java_dats    = read_gen_results('java')

Loaded 1320 record(s) for update_normal_plt-r6rs (8x31 unique) ~= 5 run(s)
Loaded 504 record(s) for update_flush_plt-r6rs (8x31 unique) ~= 2 run(s)
Loaded 600 record(s) for update_noncached_plt-r6rs (8x31 unique) ~= 2 run(s)
Loaded 364 record(s) for sw_normal_plt-r6rs (7x31 unique) ~= 1 run(s)
Loaded 245 record(s) for sw_flush_plt-r6rs (7x31 unique) ~= 1 run(s)
Loaded 602 record(s) for sw_noncached_plt-r6rs (7x31 unique) ~= 2 run(s)
Loaded 360 record(s) for res_normal_plt-r6rs (7x31 unique) ~= 1 run(s)
Loaded 245 record(s) for res_flush_plt-r6rs (7x31 unique) ~= 1 run(s)
Loaded 392 record(s) for res_noncached_plt-r6rs (7x31 unique) ~= 1 run(s)
Loaded 242 record(s) for complex_normal_plt-r6rs (11x11 unique) ~= 2 run(s)
Loaded 1 record(s) for complex_flush_plt-r6rs (1x1 unique) - no data
Loaded 1 record(s) for complex_noncached_plt-r6rs (1x1 unique) - no data
Loaded 264 record(s) for update_normal_larceny (8x31 unique) ~= 1 run(s)
Loaded 1 record(s) for update_flush_larceny (1x1 unique) 

In [13]:
java_glpk_dats = read_sol_results('java', since = datetime(2015,6,22,0,0,0))
glpk_dats      = read_sol_results('glpk')
gurobi_dats    = read_sol_results('gurobi')

Loaded 23 record(s) for update_java (1x23 unique) ~= 1 run(s)
Loaded 1 record(s) for sw_java (1x1 unique) - no data
Loaded 1 record(s) for res_java (1x1 unique) - no data
Loaded 1 record(s) for complex_java (1x1 unique) - no data
Loaded 1656 record(s) for update_glpk (8x23 unique) ~= 9 run(s)
Loaded 1 record(s) for sw_glpk (1x1 unique) - no data
Loaded 1 record(s) for res_glpk (1x1 unique) - no data
Loaded 1 record(s) for complex_glpk (1x1 unique) - no data
Loaded 216 record(s) for update_gurobi (8x27 unique) ~= 1 run(s)
Loaded 1 record(s) for sw_gurobi (1x1 unique) - no data
Loaded 1 record(s) for res_gurobi (1x1 unique) - no data
Loaded 1 record(s) for complex_gurobi (1x1 unique) - no data


In [14]:
p_ax_nr, p_line_def, p_col_def, p_label   = 0, 1, 2, 3
p_gen_racket, p_gen_larceny, p_gen_java   = 4, 5, 6
p_sol_glpk, p_sol_gurobi, p_sol_java_glpk = 4, 5, 6

In [15]:
def draw_gen(changeName, strategy, params):
    try:
        name = 'gen_{0}_{1}'.format(changeName, strategy)
        # needed number of axes equals ax_nr+1 now
        f, ax_arr = plt.subplots(nrows = ax_nr+1, ncols = 3, sharex=True, sharey=True)
        f.set_size_inches(25.5,3.5*(ax_nr+1))
        one_plot = ax_arr.shape[1] == 1

        f.patch.set_facecolor('none')
        f.patch.set_alpha(0.0)

        lines, labels = [], []
        for p in params:
            ax_tup = ax_arr if one_plot else ax_arr[p[p_ax_nr]]
            ax_j = ax_tup[0]
            ax_r = ax_tup[1]
            ax_l = ax_tup[2]
        #    ax_j.set_ylim([0,50])
        #    ax_r.set_ylim([0,10])
        #    ax_l.set_ylim([0,10])
        #    x_g = np.array(xrange(1,len(p[1])+1)) # start at one, since first gen-time is cut
            x_g = np.array(xrange(START_STEP,len(p[p_gen_racket])+START_STEP)) # start at zero
            line_java    = ax_j.plot(x_g, p[p_gen_java][0]*np.ones(len(p[p_gen_racket])), ls = p[p_line_def],
                                     c = p[p_col_def], label = p[p_label])
            line_racket  = ax_r.plot(x_g, es(p[p_gen_racket], x_g), ls = p[p_line_def],
                                     c = p[p_col_def], label = p[p_label])
            line_larceny = ax_l.plot(x_g, es(p[p_gen_larceny], x_g), ls = p[p_line_def],
                                     c = p[p_col_def], label = p[p_label])
            ax_l.legend(loc='upper left', bbox_to_anchor=(1, 1.02))
            lines.append(line_racket[0])
            labels.append(p[p_label])

        for ax in ax_arr if one_plot else ax_arr.flatten():
            ax.set_ylabel('seconds')
            ax.patch.set_alpha(1)
            ax.patch.set_facecolor('white')

        first_ax = ax_arr if one_plot else ax_arr[0]
        plt.suptitle('ILP Generation Time', fontsize = 16)
        first_ax[0].set_title('Java')
        first_ax[1].set_title('Racket')
        first_ax[2].set_title('Larceny')

        # Fine-tune figure; make subplots close to each other and hide x ticks for
        # all but bottom plots.
        f.subplots_adjust(hspace=0.2)
        plt.setp([a.get_xticklabels() for a in f.axes[:-3]], visible=False)
        plt.setp([a.get_yticklabels() for a in f.axes], visible=True)
        plt.savefig('doc/{}.pdf'.format(name), facecolor=f.get_facecolor(), edgecolor='none')
        plt.savefig('{}.png'.format(name), facecolor=f.get_facecolor(), edgecolor='none')
    except Exception as e:
        print 'Error while drawing gen in {0}-{1}: {2}'.format(change, strategy, e)
        traceback.print_exc(file=sys.stdout)
    finally:
        plt.close()

In [16]:
def draw_sol(changeName, params):
    try:
        name = 'sol_{}'.format(changeName)
        f, ax_arr = plt.subplots(nrows = ax_nr+1, ncols = 3, sharex=True, sharey=True)
        f.set_size_inches(25.5,3.5*(ax_nr+1))
        one_plot = ax_arr.shape[1] == 1

        f.patch.set_facecolor('none')
        f.patch.set_alpha(0.0)

        lines, labels = [], []
        for p in params:
            ax_tup = ax_arr if one_plot else ax_arr[p[p_ax_nr]]
            ax_javaglpk = ax_tup[0]
            ax_glpk     = ax_tup[1]
            ax_gurobi   = ax_tup[2]
            x = np.array(xrange(0,len(p[p_sol_glpk]))) # start at zero
            line_javaglpk = ax_javaglpk.plot(x, p[p_sol_java_glpk][0]*np.ones(len(p[p_sol_glpk])), ls = p[p_line_def],
                                             c = p[p_col_def], label = p[p_label])
            line_glpk     = ax_glpk.plot(x, es(p[p_sol_glpk],x), ls = p[p_line_def],
                                         c = p[p_col_def], label = p[p_label])
            line_gurobi   = ax_gurobi.plot(x, es(p[p_sol_gurobi],x), ls = p[p_line_def],
                                           c = p[p_col_def], label = p[p_label])
            ax_gurobi.legend(loc='upper left', bbox_to_anchor=(1, 1.02))
            lines.append(line_glpk[0])
            labels.append(p[p_label])

        for ax in ax_arr if one_plot else ax_arr.flatten():
            ax.set_ylabel('seconds')
            ax.patch.set_alpha(1)
            ax.patch.set_facecolor('white')

        first_ax = ax_arr if one_plot else ax_arr[0]
        plt.suptitle('ILP Solving Time', fontsize = 16)
        first_ax[0].set_title('GLPK (Java)')
        first_ax[1].set_title('GLPK (Scheme)')
        first_ax[2].set_title('Gurobi (Scheme)')

        # Fine-tune figure; make subplots close to each other and hide x ticks for
        # all but bottom plots.
        f.subplots_adjust(hspace=0.2)
        plt.setp([a.get_xticklabels() for a in f.axes[:-3]], visible=False)
        plt.setp([a.get_yticklabels() for a in f.axes], visible=True)
        plt.savefig('doc/{}.pdf'.format(name))
        plt.savefig('{}.png'.format(name))
    except Exception as e:
        print 'Error while drawing sol in {0}: {1}'.format(changeName, e)
        traceback.print_exc(file=sys.stdout)
    finally:
        plt.close()

In [17]:
def draw_comp_sol(changeName, params):
    try:
        name = 'comp_sol_{}'.format(changeName)
        indices = [3, 12, 14, 19]
        for i in indices:
            p = params[i]
            x = np.array(xrange(0,len(p[p_sol_glpk]))) # start at zero
            line_javaglpk = plt.plot(x, p[p_sol_java_glpk][0]*np.ones(len(p[p_sol_glpk])), ls = linestyles[0],
                                     c = p[p_col_def], label = p[p_label])
            line_glpk     = plt.plot(x, es(p[p_sol_glpk],x), ls = p[p_line_def], c = p[p_col_def])
            line_gurobi   = plt.plot(x, es(p[p_sol_gurobi],x), ls = p[p_line_def], c = p[p_col_def])
        plt.legend(loc = 'right')
        plt.ylabel('seconds')
        plt.suptitle('ILP Solving Time - Comparison', fontsize = 16)
        plt.savefig('doc/{}.pdf'.format(name))
        plt.savefig('{}.png'.format(name))
    except Exception as e:
        print 'Error while drawing comp-sol in {0}: {1}'.format(changeName, e)
        traceback.print_exc(file=sys.stdout)
    finally:
        plt.close()

In [18]:
## Currently disabled due to fragile and unused output
if False:
    START_STEP, MAX_PLOTS_IN_ONE = 0, 7
    for change in kinds['changes']:
        print 'Change = {}'.format(change)
        for strategy in kinds['strategies']:
            ax_nr = 0
            print 'Stategy = {}'.format(strategy)
            current_plot, ax_nr, last_res = 0, 0, -1
            gen_params = []
            for i in xrange(len(specs)):
                current_res = specs[i][2]
                current_plot += 1
                if current_plot > MAX_PLOTS_IN_ONE and last_res != current_res:
                    ax_nr += 1
                    current_plot = 0
                gen_params.append([ax_nr, line_def[i], color_def[i], '{2:d} x ({4}*{5}*{6})'.format(*specs[i]),
                                   safe(racket_dats[change][strategy],i,START_STEP),
                                   safe(larceny_dats[change][strategy],i,START_STEP),
                                   safe(java_dats[change][strategy],i)])
                last_res = current_res
            draw_gen(change, strategy, gen_params)
            # end of for strategies
        sol_params = []
        ax_nr = 0
        for i in xrange(len(specs)):
            current_res = specs[i][2]
            current_plot += 1
            if current_plot > MAX_PLOTS_IN_ONE and last_res != current_res:
                ax_nr += 1
                current_plot = 0
            sol_params.append([ax_nr, line_def[i], color_def[i], '{2:d} x ({4}*{5}*{6})'.format(*specs[i]),
                               safe(glpk_dats[change],i),
                               safe(gurobi_dats[change],i),
                               safe(java_glpk_dats[change],i)])
        draw_sol(change, sol_params)
        draw_comp_sol(change, sol_params)

## Boxplots for generation times

In [19]:
for (data, name) in ((java_dats, 'java'), (racket_dats, 'racket'), (larceny_dats, 'larceny')):
    print name+':'
    #change, strategy = 'update', 'normal'
    for change in kinds['changes']:
    #if True:
        sys.stdout.write(change+' ')
    #    if True:
        for strategy in kinds['strategies']:
            sys.stdout.write(strategy+',')
            cdat = data[change][strategy]
            plt.boxplot(cdat.transpose())
            axes = plt.gca()
            axes.set_ylim([0,50])
            plt.title('{0} {1} {2}'.format(name.title(), change.title(), strategy.title()))
            plt.ylabel('seconds')
            plt.xlabel('time')
            plt.savefig('gen_bp_{0}_{1}_{2}.png'.format(name,change,strategy))
            plt.savefig('doc/gen_bp_{0}_{1}_{2}.pdf'.format(name,change,strategy))
            plt.close()
    print ''

java:
update normal,flush,noncached,sw normal,flush,noncached,res normal,flush,noncached,complex normal,flush,noncached,
racket:
update normal,flush,noncached,sw normal,flush,noncached,res normal,flush,noncached,complex normal,flush,noncached,
larceny:
update normal,flush,noncached,sw normal,flush,noncached,res normal,flush,noncached,complex normal,flush,noncached,


## Boxplot for solving times

In [20]:
for (data, name) in ((java_glpk_dats, 'java'), (glpk_dats, 'glpk'), (gurobi_dats, 'gurobi')):
    print name+':'
    #change = 'update'
    for change in kinds['changes']:
    #if True:
        sys.stdout.write(change+' ')
    #    if True:
        cdat = data[change]
        plt.boxplot(cdat.transpose())
        plt.title('{0} {1}'.format(name.title(), change.title()))
        plt.ylabel('seconds')
        plt.xlabel('time')
        plt.savefig('sol_bp_{0}_{1}.png'.format(name,change))
        plt.savefig('doc/sol_bp_{0}_{1}_{2}.pdf'.format(name,change,strategy))
        plt.close()
    print ''

java:
update sw res complex 
glpk:
update sw res complex 
gurobi:
update sw res complex 


## Times for initial generation of the ILP

In [21]:
print reduce(lambda x,y: np.concatenate((x,y)),[racket_dats[change][strategy][0] for strategy in kinds['strategies']
                                              for change in kinds['changes']]).astype(int), '<- Racket'
print reduce(lambda x,y: np.concatenate((x,y)),[larceny_dats[change][strategy][0] for strategy in kinds['strategies']
                                          for change in kinds['changes']]).astype(int), '<- Larceny'

[ 0  0  2 15  0  0  3 21  1  1  2  5  9 10 13 13 17 22 27 20 25 30 37  2  6
 10 14  0  0  1  7  0  0  1 14  0  0  3 19  1  1  2  5  8 10 12 14 17 21 25
 19 24 28 34  3  6 10 14  0  0  1  6  0  0  1 14  0  0  3 19  1  1  2  5  8
 10 12 14 17 21 25 19 23 29 35  2  6 10 14  0  0  1  6  0  5  9 11 13 14 22
  2  7 11 15  0  0  1 15  0  0  3 22  1  1  2  5  9 10 13 14 18 22 28 20 26
 30 36  2  6 10 14  0  0  1  8  0  0  1 15  0  0  2 17  1  1  2  4  7  9 10
 11 14 18 24 16 20 25 30  2  5  9 12  0  0  1  6  0  0  1 15  0  0  2 17  1
  1  2  4  8  9 11 11 14 18 22 16 20 25 31  2  5  8 12  0  0  1  6  0  0  0
  2 17  0  0  4 27  1  3  4  8 12 15 17 20 26 30 36 28 34 40 47  4 10 17 23
  0  1  2  8  0  0  2 15  0  0  4 27  1  3  4  8 12 15 17 20 24 30 36 28 33
 40 47  4 10 16 22  0  1  2  8  0  0  2 15  0  0  4 26  1  3  4  8 12 14 17
 19 24 30 35 28 33 40 48  5 10 16 24  0  1  2  9  0] <- Racket
[ 0  0  1 11  0  0  3 22  1  2  3  6 10 12 15 18 22 27 32 25 30 38 45  4  9
 16 24  0  0  1  3  0  0 

## Attribute metrics

- X = {normal, flushed, noncached}
- (X1,X2) = {(normal, flushed), (normal, noncached), (flushed, noncached)}

In [22]:
def read_att_result(name = 'profiling/att-percentages.csv', adjust_number = True):
    def cdir(text):
        prefix = text[:-4]
        number = int(text[-3:])
        if not adjust_number or prefix == 'update':
            return number
        elif prefix == 'sw':
            return 31+number
        elif prefix == 'res':
            return 62+number
        elif prefix == 'complex':
            return 93+number
        elif prefix == '':
            pass
        else:
            print 'Unknown prefix "{0}"'.format(prefix)
    dat = np.genfromtxt(name, delimiter=',', names=True,
                        #dtype=(int, float, float, float, float, float, float, float),
                        converters={'dir': cdir})
    dat.sort(axis=0)
    # check for incomplete values
    
    print 'Loaded {0} ({1} zero-values) attribute metric run(s) from {2}'.format(
        len(dat), reduce(lambda total,row: total + reduce(lambda x,y: x+1 if y == 0 else x, row, 0),dat,0), name)
    return dat

In [23]:
att_dat = read_att_result()
att_dat.dtype

Loaded 104 (173 zero-values) attribute metric run(s) from profiling/att-percentages.csv


dtype([('dir', '<f8'), ('normalBaseline', '<f8'), ('flushedBaseline', '<f8'), ('noncachedBaseline', '<f8'), ('ratioNormalToFlushed', '<f8'), ('ratioNormalToNoncached', '<f8'), ('ratioFlushedToNoncached', '<f8'), ('speedupNormalToFlushed', '<f8'), ('speedupNormalToNoncached', '<f8'), ('speedupFlushedToNoncached', '<f8')])

1) {X}basline: `total.X.computed / total.X.called`
- the baseline of the method X

In [24]:
width = 0.5
x = att_dat['dir']
for n, (l, c) in enumerate((('noncached', 'grey'), ('flushed', 'g'), ('normal', 'b'))):
    y = att_dat[l + 'Baseline']
    #plt.plot(x, y, label = l, color = c)
    plt.bar(x*4*width + n*width, y, width, label = l, color = c)

plt.legend(loc = 'best')
plt.ylabel('% computed')
plt.suptitle('Attribute Baselines', fontsize = 16)
plt.savefig('doc/att_bl.pdf')
!pdfcrop doc/att_bl.pdf doc/att_bl_cropped.pdf > /dev/null
plt.savefig('att_bl.png')
plt.close()

2) ratio{X1}To{X2}: `total.{X1}.computed / total.{X2}.called`
- the efficiency of the incremental approach in comparison to the method X2,
  i.e. the ratio between actual work done in X1 compared to possible work done with method X2

In [25]:
x = att_dat['dir']
for n, (name, c, ls) in enumerate((('FlushedToNoncached', 'orange', '--'),
                            ('NormalToFlushed', 'purple', '-'),
                            ('NormalToNoncached', 'r', '-.'))):
    y = att_dat['ratio' + name]
    plt.plot(x, y, label = name.replace('To', r'$\rightarrow$'), c = c, ls = ls)
    #plt.bar(x*3*width + n*width, y, width, label = l, color = c)

plt.legend(loc = 'best')
plt.ylabel('%')
plt.suptitle('Attribute Ratios', fontsize = 16)
plt.savefig('doc/att_r.pdf')
!pdfcrop doc/att_r.pdf doc/att_r_cropped.pdf > /dev/null
plt.savefig('att_r.png')
plt.close()

3) speedup{X1}To{X2}: `(total.{X2}.computed / total.{X2}.called) - (total.{X1}.computed / total.{X2}.called)`
- = `baseline({X2}) - ratio({X1}, {X2})`
- the "speed-up" of the incremental approach (normal or flushed) in comparison to the method X2

In [26]:
x = att_dat['dir']
for n, (l, c) in enumerate((('NormalToNoncached', 'r'),
                            ('FlushedToNoncached', 'orange'),
                            ('NormalToFlushed', 'yellow'))):
    y = att_dat['speedup' + l]
    plt.plot(x, y, label = l.replace('To', r'$\rightarrow$'), color = c)
    #plt.bar(x*3*width + n*width, y, width, label = l, color = c)

plt.legend(loc = 'best')
plt.ylabel('%')
plt.suptitle('Attribute Speed-Ups', fontsize = 16)
plt.savefig('doc/att_sp.pdf')
!pdfcrop doc/att_sp.pdf doc/att_sp_cropped.pdf > /dev/null
plt.savefig('att_sp.png')
plt.close()

## Boxplots for attribute measures

In [27]:
att_totals = {}
for change in kinds['changes']:
    att_totals[change] = read_att_result(name = 'profiling/splitted/att-percentages_{}.csv'.format(change),
                                         adjust_number = False)

Loaded 31 (0 zero-values) attribute metric run(s) from profiling/splitted/att-percentages_update.csv
Loaded 31 (0 zero-values) attribute metric run(s) from profiling/splitted/att-percentages_sw.csv
Loaded 31 (85 zero-values) attribute metric run(s) from profiling/splitted/att-percentages_res.csv
Loaded 11 (88 zero-values) attribute metric run(s) from profiling/splitted/att-percentages_complex.csv


In [28]:
PRINT_NONCACHED = True

In [29]:
def ratio_columns(name):
    if name == 'normal':
        return 'ratioNormalToFlushed'
    if name == 'flushed':
        return 'ratioNormalToNoncached'
    if name == 'noncached':
        return 'ratioFlushedToNoncached'
    raise Exception('Unknown name: {}'.format(name))
def ratio_speedup_names(name):
    if name == 'normal':
        return r'nm$\rightarrow$fs'
    if name == 'flushed':
        return r'nm$\rightarrow$nc'
    if name == 'noncached':
        return r'fs$\rightarrow$nc'
    raise Exception('Unknown name: {}'.format(name))
def speedup_columns(name):
    if name == 'normal':
        return 'speedupNormalToFlushed'
    if name == 'flushed':
        return 'speedupNormalToNoncached'
    if name == 'noncached':
        return 'speedupFlushedToNoncached'
    raise Exception('Unknown name: {}'.format(name))

In [30]:
def draw_att(fcolumn, flabel, title, fname_part, rotation_xaxis = None):
    """ Draw boxplots for attribute
    @param:fcolum: function with one parameter (name) returning column name
    @param:flabel: function with one parameter (name) returning label
    @param:title: title of plot
    @param:fname_part: unique filename part for storing plot
    @param:rotation_xaxis: rotation of xaxis ticks (default: no rotation)"""
    try:
        fig, axes = plt.subplots(ncols=len(kinds['changes']))
#        if not rotation_xaxis:
#            print dir(fig)
        fig.suptitle(title, fontsize = 16)
        l = ['normal', 'flushed']
        if PRINT_NONCACHED:
            l.append('noncached')
        for i, change in enumerate(kinds['changes']):
            att_dat = att_totals[change]
            axes[i].boxplot([att_dat[fcolumn(name)] for name in l], labels = [flabel(name) for name in l])
            axes[i].set_title(change, fontsize=10)
            if i > 0:
                axes[i].set_yticklabels([])
        for ax in axes.flatten():
            if PRINT_NONCACHED:
                ax.set_ylim([0,1.05])
            if rotation_xaxis:
                plt.setp(ax.xaxis.get_majorticklabels(), rotation=rotation_xaxis)
#        plt.axes().xaxis.set_major_locator(matplotlib.ticker.FixedLocator(range(0, 3, 2)))
#        plt.axes().xaxis.set_minor_locator(matplotlib.ticker.FixedLocator([1]))
#        plt.axes().xaxis.set_minor_formatter(matplotlib.ticker.FormatStrFormatter("%s"))
#        plt.axes().tick_params(which='major', pad=20, axis='x')
        plt.subplots_adjust(top=0.85)
        plt.savefig('doc/{}.pdf'.format(fname_part))
        !pdfcrop {'doc/{}.pdf'.format(fname_part)} {'doc/{}_cropped.pdf'.format(fname_part)} > /dev/null
        plt.savefig('{}.png'.format(fname_part))
    except Exception as e:
        print 'Error while drawing att in {0}: {1}'.format(change, e)
        traceback.print_exc(file=sys.stdout)
    finally:
        plt.close()

In [31]:
draw_att(lambda x: '{}Baseline'.format(x), lambda x: x[0]+x[3], 'Attribute Baselines', 'att_box_bl')
draw_att(ratio_columns, ratio_speedup_names, 'Attribute Ratios', 'att_box_r', rotation_xaxis = 30)
draw_att(speedup_columns, ratio_speedup_names, 'Attribute Speedups', 'att_box_sp', rotation_xaxis = 30)